In [1]:
%run stdPackages.ipynb # this imports a lot of useful packages 
import base

In [11]:
#Define Variables for Consumers, producers etc. 

MWP  = pd.Series([10, 20], index = pd.Index(['Consumer 1','Consumer 2'], name = 'c'))
D̄    = pd.Series([0.5, 0.5], index = pd.Index(['Consumer 1','Consumer 2'], name = 'c'))
mc   = pd.Series([15, 5], index = pd.Index(['Conv. plant','Wind turbine'], name = 'id'))
q_id = pd.Series([0.5, 0.5], index = pd.Index(['Conv. plant', 'Wind turbine'], name = 'id'))
γ    = pd.Series([1, 1, 1, 0.5], index = pd.MultiIndex.from_tuples([('Conv. plant',1), ('Wind turbine',1),
                                                                    ('Conv. plant',2), ('Wind turbine', 2)],
                                                                   names = ['id','h']))
q    = q_id * γ
FOM = pd.Series([2, 3], index = pd.Index(['Conv. plant', 'Wind tubine'], name= 'id'))

In our model, we are solving for a couple of variables. x, e first  

In [3]:
#Define stacks for optimisation
c = np.hstack([mc, mc, -MWP, -MWP])
l = np.zeros(8)
u = np.hstack([q, D̄, D̄])
pd.DataFrame({'c':c, 'l':l, 'u':u}) # show the vectors here

,c,l,u
0,15,0.0,0.50
1,5,0.0,0.50
2,15,0.0,0.50
3,5,0.0,0.25
4,-10,0.0,0.50
5,-20,0.0,0.50
6,-10,0.0,0.50
7,-20,0.0,0.50


In [4]:
#Create database to input data
db = pyDbs.SimpleDB() # initialize empty database

In [6]:
#Data stuff here
db['mc'] = mc
db['D̄']  = D̄
db['MWP']= MWP
db['q']  = q

In [7]:
#Debug database
print('Symbols registered in the database: ', db.symbols.keys()) # The symbols in the database are stored as keys
                                                                 # in the central dictionary, accessed by db.symbols
print('Example symbol "q": \n\n', db['q'], '\n')  # The values associated with a symbol can be viewed using a dictionary-style syntax
pyDbs.readSets(db)  # This function adds the unique indices in each of the symbol series as separate symbols to the db
print('Symbols including the new sets: ', db.symbols.keys())
print('Values in the "id" set: ', db['id']) # Now we can access the index values using the same syntax

Symbols registered in the database:  dict_keys(['mc', 'D̄', 'MWP', 'q'])
Example symbol "q": 

 id            h
Conv. plant   1    0.50
Wind turbine  1    0.50
Conv. plant   2    0.50
Wind turbine  2    0.25
dtype: float64 

Symbols including the new sets:  dict_keys(['mc', 'D̄', 'MWP', 'q', 'id', 'c', 'h'])
Values in the "id" set:  Index(['Conv. plant', 'Wind turbine'], dtype='object', name='id')


In [8]:
db.updateAlias(alias = [('h','h_constr')]) # define h_constr as a copy of the index 'h'.

In [9]:
globalDomains = {'E':   pd.MultiIndex.from_product([db['id'], db['h']]),
                 'D': pd.MultiIndex.from_product([db['c'], db['h']]),
                 'equilibrium':  db['h_constr']}
globalDomains

{'E': MultiIndex([( 'Conv. plant', 1),
             ( 'Conv. plant', 2),
             ('Wind turbine', 1),
             ('Wind turbine', 2)],
            names=['id', 'h']),
 'D': MultiIndex([('Consumer 1', 1),
             ('Consumer 1', 2),
             ('Consumer 2', 1),
             ('Consumer 2', 2)],
            names=['c', 'h']),
 'equilibrium': Int64Index([1, 2], dtype='int64', name='h_constr')}